In [ ]:
# Markov with Syllable Counter
# Same rhyme ranker as MikesVersion1
# Changelog: 
# - Can specify syllables per line

In [ ]:
#@title Import Statements
!pip install PyGithub
!pip install pyphen

# Package Imports
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
from pathlib import Path # The Path class
import pyphen

# For the more advanced requests
import base64
import os
import sys
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass

     |████████████████████████████████| 296kB 4.0MB/s 
     |████████████████████████████████| 1.9MB 5.1MB/s 


In [ ]:
help(pyphen)

Help on package pyphen:

NAME
    pyphen

DESCRIPTION
    Pyphen
    
    Pure Python module to hyphenate text, inspired by Ruby's Text::Hyphen.

PACKAGE CONTENTS


CLASSES
    builtins.object
        Pyphen
    
    class Pyphen(builtins.object)
     |  Hyphenation class, with methods to hyphenate strings in various ways.
     |  
     |  Methods defined here:
     |  
     |  __call__ = iterate(self, word)
     |  
     |  __init__(self, filename=None, lang=None, left=2, right=2, cache=True)
     |      Create an hyphenation instance for given lang or filename.
     |      
     |      :param filename: filename of hyph_*.dic to read
     |      :param lang: lang of the included dict to use if no filename is given
     |      :param left: minimum number of characters of the first syllabe
     |      :param right: minimum number of characters of the last syllabe
     |      :param cache: if ``True``, use cached copy of the hyphenation patterns
     |  
     |  inserted(self, word, hyph

In [ ]:
#@title Function Definitions
# Recursively Import the Data (AUTOMATIC)

def _decode_and_write(file__, path_):
    data = file__.decoded_content
    data = data.decode('utf-8')[1:]
    with open(path_, 'w') as writefile:
        writefile.write(data) 
    data = data.splitlines()
    data_rows = []
    for count, word in enumerate(data):
        if count>0:
            data_rows.append(word.split(','))
    data = pd.DataFrame(data_rows)
    data = data.to_numpy()
    return data


def import_github(path_name="AllLyrics.txt"):
    """
    Function for importing the github file
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()
    print("Importing Github cleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    RAP_DATA = []
    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the Lyrics
            RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    
    temp_path = Path(path_name)
    if temp_path.is_file(): 
        if os.stat(path_name).st_size == 0:
            write_bool2 = True
        else: 
            write_bool2 = False
    else: 
        write_bool2 = True
    
    if write_bool2: 
        for lyric in RAP_DATA: 
            try:
                with open(path_name, 'w') as writefile: 
                    writefile.write(lyric)
            except: 
                print("Error, file moved/deleted during write")
        print("{} is now up to date!".format(path_name))
    else: 
        print("{} is already up to date!".format(path_name))
    
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 

        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        path = path[title_start + 6:title_start + title_len + 4]

        print("Writing file {} {}".format(counter, path))
        temp_path = Path(path)
        if temp_path.is_file():
            with open(path,'w'): pass # Cheeky way to clear the file if it exists
        
        # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy array 
        data = file_.decoded_content
        data = data.decode('utf-8')[1:]

        with open(path, 'w') as writefile:
            writefile.write(data) 
        print("All files now up to date!")


def update_github(write_bool=False, path_name="AllLyrics.txt"):
    """
    Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
    write_bool: bool
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()

    print("Importing editing csv files...")

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path 
        path = str(path)
        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len + 4]
        print("Writing file {} {}".format(counter, name))
        if name.lower() == "censors.csv":
            censors = _decode_and_write(file_, path)
        elif name.lower() == "capitals.csv":
            capitals = _decode_and_write(file_, path)
        else: 
            _decode_and_write(file_, path)
    print("All editing csv files are up to date!")

    print("Importing Github uncleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/UNCLEAN", ref=branch)

    RAP_DATA = []
    rap_lyric_names = []

    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('UNCLEAN')
            title_len = path[title_start:].find('.')
            name = path[title_start + 8:title_start + title_len]
            if name[-2:] == 'UC':
                name = name[:-2]
            rap_lyric_names.append(name) 

        # Append the Lyrics
        RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
        
    # Remove the \ufeff at the beginning O(n)
    for count, lyric in enumerate(RAP_DATA): 
        RAP_DATA[count] = lyric[1:]

    # Censor the profanities O(n*m + n*m2) m > m2 xor m2 > m
    for count in range(len(RAP_DATA)): 
        for i in range(len(censors[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))
        for i in range(len(capitals[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(capitals[i, 0]), str(capitals[i, 1]))

    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    cleaned_names = []
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 
        print("File {} ".format(counter + 1) + path)
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('CLEAN')
            title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len]
        if name[-2:] == 'CL':
            name = name[:-2]
        cleaned_names.append(name) 

    # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
    # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
    # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
    # print(rap_lyric_names)
    # print(cleaned_names)
    print("Committing files to github...")
    for counter, new_name in enumerate(rap_lyric_names): 
        if new_name in cleaned_names: 
            duplicate = r_proj_clone.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
            r_proj_clone.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
        else:
            r_proj_clone.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

    if write_bool: 
        print("Writing text file to: {}".format(path_name))
        with open(path_name, 'w') as writefile:
            for lyric in RAP_DATA:
                writefile.write(lyric)


In [ ]:
count = int(input("How many syllables per line? "))

How many syllables per line? 16


In [ ]:
dic = pyphen.Pyphen(lang='en_EN') # set pyphen dictionary to English

def generate_rap(Vocabulary):
    start_line = line_generator(Vocabulary)
    all_other_lines = [line_generator(Vocabulary) for i in range(999)]
    rap = [start_line]
    
    for i in range (19):
        next_line = next_line_stop_count(rap[len(rap) - 1], all_other_lines)
        all_other_lines.remove(next_line)
        rap.append(next_line)
    return rap


# Generate text
def line_generator(Vocabulary):
    index = 1
    chain = {}
    # count = 16 # https://colemizestudios.com/rap-lyrics-syllables/, apparently rappers usually use semiquavers
    linecount = 0
    
    for word in Vocabulary[index:]:
        key = Vocabulary[index-1]
        if key in chain:
            chain[key].append(word)
        else:
            chain[key] = [word]
        index += 1
        
    word1 = random.choice(list(chain.keys()))
    line = word1.capitalize()
    wordsyllables = dic.inserted(word1)
    wordcount = len(wordsyllables.split('-'))
    linecount += wordcount

    while linecount <= count:
        word2 = random.choice(chain[word1])
        word1 = word2
        line += ' ' + word2.lower()
        wordsyllables = dic.inserted(word2)
        wordcount = len(wordsyllables.split('-'))
        linecount += wordcount
    return line


# Rhyme Functions
def reverse_syllable_extract(text):
    sy_form = []
    characters = [char for char in text]
    sylls = ['a', 'e', 'i', 'o', 'u']
    for x in characters:
        if x in sylls:
            sy_form.append(x)
    sy_form.reverse()
    return sy_form


def rev_syllable_stop_count(text1, text2):
    count = True 
    i = 0
    counter = 0
    syll1 = reverse_syllable_extract(text1)
    syll2 = reverse_syllable_extract(text2)
    while count == True:
        if i < min(len(syll1), len(syll2)) and syll1[i] == syll2[i]:
            counter += 1
            i += 1
        else:
            count = False
    return counter


def next_line_stop_count(start_line, lines):
    sy_lines = []
    for i in lines:
        sy_lines.append(rev_syllable_stop_count(start_line, i))
    choice = sy_lines[0]
    count = 0
    for i in range(len(sy_lines)):
        if sy_lines[i] > choice:
            choice = sy_lines[i]
    return lines[sy_lines.index(choice)]

In [ ]:
# Import all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
import_github()

Enter your PAT key ··········
Main Branch: Yes or No? No
Importing Github cleaned text files...
AllLyrics.txt is now up to date!
Writing file 0 capitals.csv
All files now up to date!
Writing file 1 censors.csv
All files now up to date!


In [ ]:
# Extract all of Mike's lyrics. 
Text = open("AllLyrics.txt", "r").read()
Vocabulary = ''.join([i for i in Text if not i.isdigit()]).replace("\n", " ").split(' ')
# print(Vocabulary)
# line_generator(Vocabulary)

In [ ]:
for line in generate_rap(Vocabulary):
    print(line)

Play ya even groups like like that god damn thing she came to give you to feel
Daily recruitin theres a job hit the frick out me please dont wanna do when they
Disrespect me a ninja ninja money hoes man rockin on even had a glock when they
Ds crept in time im lookin like to the thigh the telly we gon do seven
Cigars in blue and you thought so she used to matteo now who got these
Gun up put bombs in your motherfrickin fun when yall lungs we got to go there
Lets ride lets leave i cant you did i love with yayo peel more problems we
Rockin on the road i mean i aint my pocket thats on hoes dont let let
Rodeos flash the king of all the one gives me no ifs ands or the beds completely
Puttin karats in the blunt aint no doubt the park pissy off and violence
Doubleoh with all we just hypnotize me big poppa to all the third power he
Porno flicks b***** the one phone to do to be handlin with long still tote tecs
Venice beach sippin white owls and dine who you frick with benjamins under the
Hows ev

In [ ]:
# Now move to generate a 20 line rap
# Will generate 1000 lines using line_generator then from first generated line pick next best line by rhyme 19 times
# I've left it as a list with strings as the lines, we can have fun improving things if we think this is a good start
# The above has taken ages to load as has made 1000 lines then compared each one to try and find the best next!
# Also the vocab is much longer now